In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
df = pd.read_csv('./Brutus data/plummer_triples_L0_00_i1775_e90_Lw392.csv')

forward_trajectory = df[df['Phase'] == 1]
backward_trajectory = df[df['Phase'] == -1]

total_timesteps = len(df['Timestep'].unique())
midpoint = total_timesteps // 2  # Midpoint is 4702 (if total timesteps are 9405)

particles = df['Particle Number'].unique()


delta_2 = 0

# Loop over each particle to calculate the phase space difference
for p in particles:
    forward_p = forward_trajectory[forward_trajectory['Particle Number'] == p]
    backward_p = backward_trajectory[backward_trajectory['Particle Number'] == p]
    
    # Include only up to the midpoint for forward trajectory
    forward_p = forward_p[forward_p['Timestep'] <= midpoint]
    
    # Include only up to the midpoint-1 for backward trajectory (exclude the flipped timestep)
    backward_p = backward_p[backward_p['Timestep'] < midpoint]
    
    # Ensure that the lengths now match
    # assert len(forward_p) == len(backward_p), f"Length mismatch for particle {p}: forward({len(forward_p)}) != backward({len(backward_p)})"
    
    x_f, y_f, z_f = forward_p['X Position'].values, forward_p['Y Position'].values, forward_p['Z Position'].values
    x_b, y_b, z_b = backward_p['X Position'].values, backward_p['Y Position'].values, backward_p['Z Position'].values

    vx_f, vy_f, vz_f = forward_p['X Velocity'].values, forward_p['Y Velocity'].values, forward_p['Z Velocity'].values
    vx_b, vy_b, vz_b = backward_p['X Velocity'].values, backward_p['Y Velocity'].values, backward_p['Z Velocity'].values
    
    diff_vel = (vx_f - vx_b)**2 + (vy_f - vy_b)**2 + (vz_f - vz_b)**2
    diff_pos = (x_f - x_b)**2 + (y_f - y_b)**2 + (z_f - z_b)**2
    
    # Sum over all phase-space coordinates for this particle
    diff = diff_vel + diff_pos
    
    # Sum over all timesteps for this particle and add to total delta_2
    delta_2 += np.sum(diff)

# Delta^2 summed over all particles
# print(f"Δ² = {delta_2}")

# Delta between initial and final states


Δ² = 4842706.7648106795


In [19]:

# Separate forward and backward trajectories
forward_trajectory = df[df['Phase'] == 1]
backward_trajectory = df[df['Phase'] == -1]

total_timesteps = len(df['Timestep'].unique())
midpoint = total_timesteps // 2  # Midpoint is 4702 (if total timesteps are 9405)

particles = df['Particle Number'].unique()

# Function to compute phase space distance Δ² between two states
def compute_delta_2(forward_state, backward_state):
    # Extract positions
    x_f, y_f, z_f = forward_state['X Position'].values, forward_state['Y Position'].values, forward_state['Z Position'].values
    x_b, y_b, z_b = backward_state['X Position'].values, backward_state['Y Position'].values, backward_state['Z Position'].values

    # Extract velocities
    vx_f, vy_f, vz_f = forward_state['X Velocity'].values, forward_state['Y Velocity'].values, forward_state['Z Velocity'].values
    vx_b, vy_b, vz_b = backward_state['X Velocity'].values, backward_state['Y Velocity'].values, backward_state['Z Velocity'].values

    # Compute the phase-space distance for this timestep
    diff_vel = (vx_f - vx_b)**2 + (vy_f - vy_b)**2 + (vz_f - vz_b)**2
    diff_pos = (x_f - x_b)**2 + (y_f - y_b)**2 + (z_f - z_b)**2
    
    return np.sum(diff_vel + diff_pos)

# Delta_2 for initial and final states
delta_2_initial_final = 0

# Check for a single particle and inspect the values
for p in particles[:1]:  # Let's check just for one particle for now
    forward_p = forward_trajectory[forward_trajectory['Particle Number'] == p]
    backward_p = backward_trajectory[backward_trajectory['Particle Number'] == p]
    
    # Extract initial (Timestep == 0)
    forward_initial = forward_p[forward_p['Timestep'] == 0]
    
    # Extract the final timestep available in backward trajectory
    max_backward_timestep = backward_p['Timestep'].max()
    backward_final = backward_p[backward_p['Timestep'] == max_backward_timestep]
    
    # Print out the states for debugging
    print(f"Forward Initial State (Particle {p}):\n", forward_initial)
    print(f"Backward Final State (Particle {p}):\n", backward_final)
    print(f"Max timestep in backward trajectory for Particle {p}: {max_backward_timestep}")
    
    # Compute the delta² for the initial and final state
    delta_2_initial_final += compute_delta_2(forward_initial, backward_final)

# Output the result
print(f'Delta² between initial and final states: {delta_2_initial_final}')
#print(f'Delta² after each integration step: {delta_2_per_step}')


# Compute amplification factor for each integration step
#A = [np.sqrt(delta_2_initial_final / delta_2_step) for delta_2_step in delta_2_per_step]

# Plot the evolution of A over time
""" plt.plot(A)
plt.xlabel('Integration Step')
plt.ylabel('Amplification Factor')
plt.title('Amplification Factor vs. Integration Step')
plt.show() """

Forward Initial State (Particle 1):
    Timestep  Particle Number      Mass  X Position  Y Position  Z Position  \
0       0.0                1  0.333333    0.954551   -1.967417    0.464114   

   X Velocity  Y Velocity  Z Velocity  Phase  
0         0.0        -0.0         0.0      1  
Backward Final State (Particle 1):
        Timestep  Particle Number      Mass  X Position  Y Position  \
28212    4702.0                1  0.333333    0.854793   -2.728481   

       Z Position  X Velocity  Y Velocity  Z Velocity  Phase  
28212    0.368119    0.042382   -0.155171    0.017275     -1  
Max timestep in backward trajectory for Particle 1: 4702.0
Delta² between initial and final states: 0.624557791675682


" plt.plot(A)\nplt.xlabel('Integration Step')\nplt.ylabel('Amplification Factor')\nplt.title('Amplification Factor vs. Integration Step')\nplt.show() "